In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from ml_metrics import rmse
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston

In [2]:
boston = load_boston()
bos = pd.DataFrame(boston.data)
bos.columns = boston.feature_names
bos['MEDV'] = boston.target

## Fit a linear regressor to the data as a baseline.  

In [3]:
train_set = bos.sample(frac=0.7, random_state=100)
test_set = bos[~bos.isin(train_set)].dropna()

In [4]:
X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

In [5]:
reg = LinearRegression(normalize=True).fit(X_train, Y_train)
print(reg.score(X_train, Y_train))
print(reg.coef_)
print(reg.intercept_)
print(reg.get_params())

0.7478284701218886
[-1.35456753e-01  5.48606010e-02  5.46611167e-02  3.57648807e+00
 -2.01163242e+01  3.96567027e+00  1.33685712e-02 -1.48716658e+00
  2.99295349e-01 -9.83868843e-03 -9.45023886e-01  6.45207267e-03
 -5.77572297e-01]
36.0793476882823
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': True}


What is the loss and what are the goodness of fit parameters?

In [6]:
print("The loss of the model is {}.".format(reg.score(X_train, Y_train)))
for k, v in reg.get_params().items():
    print("Goodness of fit parameter {} has a value of {}".format(k,v))

The loss of the model is 0.7478284701218886.
Goodness of fit parameter copy_X has a value of True
Goodness of fit parameter fit_intercept has a value of True
Goodness of fit parameter n_jobs has a value of None
Goodness of fit parameter normalize has a value of True


In [7]:
Y_pred = reg.predict(X_test)

orig_mae = mean_absolute_error(Y_test,Y_pred)
orig_mse = mean_squared_error(Y_test,Y_pred)
orig_rmse_val = rmse(Y_test,Y_pred)
orig_r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%orig_mae)
print("MSE:  %.3f"%orig_mse)
print("RMSE:  %.3f"%orig_rmse_val)
print("R2:  %.3f"%orig_r2)

MAE: 3.605
MSE:  24.099
RMSE:  4.909
R2:  0.705


In [8]:
res_frame = pd.DataFrame({'data':'original',
                   'imputation':'none',
                   'mae': orig_mae, 
                   'mse': orig_mse, 
                   'rmse':orig_rmse_val, 
                   'R2':orig_r2,
                   'mae_diff':np.nan,
                   'mse_diff':np.nan,
                   'rmse_diff':np.nan,
                   'R2_diff':np.nan}, index=[0])
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.70494,NaN,NaN,NaN,NaN


In [9]:
train_index = train_set.index.values.astype(int)
test_index = test_set.index.values.astype(int)

Select 1%, 5% 10%, 20%, 33%, and 50% of your data in a single column [hold that column selection constant throughout all iterations] (Completely at random), replace the original value with a NaN (i.e., “not a number” – ex., np.nan) and then perform an imputation for the missing values.   

In [10]:
in_sample = bos.sample(frac=0.01, random_state=99)
out_sample = bos[~bos.isin(in_sample)].dropna()
in_sample['AGE'] = np.nan
in_sample['AGE'] = in_sample['AGE'].fillna(out_sample['AGE'].median())
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()

In [11]:
train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]

In [12]:
X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

In [13]:
reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

0.7477990218800383
[-1.35495603e-01  5.46769124e-02  5.59365305e-02  3.58336026e+00
 -2.00572242e+01  3.97115055e+00  1.28757977e-02 -1.48614678e+00
  2.99858619e-01 -9.89226289e-03 -9.44220284e-01  6.46707615e-03
 -5.76184693e-01]
36.00239693382321
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}


In [14]:
Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

MAE: 3.610
MSE:  24.097
RMSE:  4.909
R2:  0.705


In [15]:
temp_frame = pd.DataFrame({'data':'1% imputed',
                   'imputation':'AGE',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

In [16]:
res_frame = pd.concat([res_frame, temp_frame])
res_frame

,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,1% imputed,AGE,3.610206,24.097314,4.908902,0.704954,0.005634,-0.001191,-0.000121,0.000015


In [17]:
in_sample = bos.sample(frac=0.05, random_state=99)
out_sample = bos[~bos.isin(in_sample)].dropna()
in_sample['AGE'] = np.nan
in_sample['AGE'] = in_sample['AGE'].fillna(out_sample['AGE'].median())
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()

train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]

X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

temp_frame = pd.DataFrame({'data':'5% imputed',
                   'imputation':'AGE',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

res_frame = pd.concat([res_frame, temp_frame])
res_frame

0.7474054150918357
[-1.35479447e-01  5.39751912e-02  5.68455747e-02  3.64818312e+00
 -1.95450772e+01  4.02186229e+00  5.81560155e-03 -1.51361766e+00
  2.97374435e-01 -9.83852440e-03 -9.37852993e-01  6.56772095e-03
 -5.66318324e-01]
35.71298478523523
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}
MAE: 3.603
MSE:  23.959
RMSE:  4.895
R2:  0.707


,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,1% imputed,AGE,3.610206,24.097314,4.908902,0.704954,0.005634,-0.001191,-0.000121,0.000015
0,5% imputed,AGE,3.602697,23.959056,4.894799,0.706647,-0.001874,-0.139449,-0.014224,0.001707


In [18]:
in_sample = bos.sample(frac=0.1, random_state=99)
out_sample = bos[~bos.isin(in_sample)].dropna()
in_sample['AGE'] = np.nan
in_sample['AGE'] = in_sample['AGE'].fillna(out_sample['AGE'].median())
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()

train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]

X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

temp_frame = pd.DataFrame({'data':'10% imputed',
                   'imputation':'AGE',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

res_frame = pd.concat([res_frame, temp_frame])
res_frame

0.7475901636386664
[-1.35704898e-01  5.42248059e-02  5.65665563e-02  3.61841897e+00
 -1.98062823e+01  3.99558651e+00  9.59472574e-03 -1.50029324e+00
  2.99036913e-01 -9.87201825e-03 -9.42429391e-01  6.52870503e-03
 -5.70192288e-01]
35.85538686739957
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}
MAE: 3.607
MSE:  24.040
RMSE:  4.903
R2:  0.706


,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,1% imputed,AGE,3.610206,24.097314,4.908902,0.704954,0.005634,-0.001191,-0.000121,0.000015
0,5% imputed,AGE,3.602697,23.959056,4.894799,0.706647,-0.001874,-0.139449,-0.014224,0.001707
0,10% imputed,AGE,3.606641,24.039858,4.903046,0.705658,0.002070,-0.058647,-0.005977,0.000718


In [19]:
in_sample = bos.sample(frac=0.2, random_state=99)
out_sample = bos[~bos.isin(in_sample)].dropna()
in_sample['AGE'] = np.nan
in_sample['AGE'] = in_sample['AGE'].fillna(out_sample['AGE'].median())
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()

train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]

X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

temp_frame = pd.DataFrame({'data':'20% imputed',
                   'imputation':'AGE',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

res_frame = pd.concat([res_frame, temp_frame])
res_frame

0.7475244246396398
[-1.35668627e-01  5.42339417e-02  5.75158074e-02  3.64277357e+00
 -1.97135946e+01  4.00820309e+00  8.16071048e-03 -1.51158118e+00
  2.98373506e-01 -9.89070337e-03 -9.40085350e-01  6.52989253e-03
 -5.67554567e-01]
35.78364265172914
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}
MAE: 3.613
MSE:  24.018
RMSE:  4.901
R2:  0.706


,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,1% imputed,AGE,3.610206,24.097314,4.908902,0.704954,0.005634,-0.001191,-0.000121,0.000015
0,5% imputed,AGE,3.602697,23.959056,4.894799,0.706647,-0.001874,-0.139449,-0.014224,0.001707
0,10% imputed,AGE,3.606641,24.039858,4.903046,0.705658,0.002070,-0.058647,-0.005977,0.000718
0,20% imputed,AGE,3.613030,24.017633,4.900779,0.705930,0.008459,-0.080872,-0.008244,0.000990


In [20]:
in_sample = bos.sample(frac=0.33, random_state=99)
out_sample = bos[~bos.isin(in_sample)].dropna()
in_sample['AGE'] = np.nan
in_sample['AGE'] = in_sample['AGE'].fillna(out_sample['AGE'].median())
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()

train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]

X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

temp_frame = pd.DataFrame({'data':'33% imputed',
                   'imputation':'AGE',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

res_frame = pd.concat([res_frame, temp_frame])
res_frame

0.747504950677319
[-1.35315559e-01  5.33919444e-02  5.89311604e-02  3.64446061e+00
 -1.96292875e+01  4.02686961e+00  7.67041127e-03 -1.50620066e+00
  2.98345141e-01 -9.83261044e-03 -9.44144893e-01  6.53803679e-03
 -5.65129040e-01]
35.626593631326884
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}
MAE: 3.616
MSE:  24.027
RMSE:  4.902
R2:  0.706


,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,1% imputed,AGE,3.610206,24.097314,4.908902,0.704954,0.005634,-0.001191,-0.000121,0.000015
0,5% imputed,AGE,3.602697,23.959056,4.894799,0.706647,-0.001874,-0.139449,-0.014224,0.001707
0,10% imputed,AGE,3.606641,24.039858,4.903046,0.705658,0.002070,-0.058647,-0.005977,0.000718
0,20% imputed,AGE,3.613030,24.017633,4.900779,0.705930,0.008459,-0.080872,-0.008244,0.000990
0,20% imputed,AGE,3.616309,24.027006,4.901735,0.705815,0.011738,-0.071499,-0.007288,0.000875


In [21]:
in_sample = bos.sample(frac=0.5, random_state=99)
out_sample = bos[~bos.isin(in_sample)].dropna()
in_sample['AGE'] = np.nan
in_sample['AGE'] = in_sample['AGE'].fillna(out_sample['AGE'].median())
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()

train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]

X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

temp_frame = pd.DataFrame({'data':'50% imputed',
                   'imputation':'AGE',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

res_frame = pd.concat([res_frame, temp_frame])
res_frame

0.7473373503518663
[-1.35809798e-01  5.31403869e-02  5.70717678e-02  3.68536976e+00
 -1.90691241e+01  4.07694384e+00 -3.45722949e-03 -1.54799454e+00
  2.94339306e-01 -9.76870569e-03 -9.31654051e-01  6.61033432e-03
 -5.57201038e-01]
35.65787483295118
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}
MAE: 3.601
MSE:  23.889
RMSE:  4.888
R2:  0.708


,data,imputation,mae,mse,rmse,R2,mae_diff,mse_diff,rmse_diff,R2_diff
0,original,none,3.604571,24.098505,4.909023,0.704940,NaN,NaN,NaN,NaN
0,1% imputed,AGE,3.610206,24.097314,4.908902,0.704954,0.005634,-0.001191,-0.000121,0.000015
0,5% imputed,AGE,3.602697,23.959056,4.894799,0.706647,-0.001874,-0.139449,-0.014224,0.001707
0,10% imputed,AGE,3.606641,24.039858,4.903046,0.705658,0.002070,-0.058647,-0.005977,0.000718
0,20% imputed,AGE,3.613030,24.017633,4.900779,0.705930,0.008459,-0.080872,-0.008244,0.000990
0,20% imputed,AGE,3.616309,24.027006,4.901735,0.705815,0.011738,-0.071499,-0.007288,0.000875
0,50% imputed,AGE,3.601175,23.888715,4.887608,0.707508,-0.003396,-0.209790,-0.021414,0.002569


Take two columns and create data “Missing at Random” when controlled for a third variable (i.e., if Variable Z is > 30, then Variables X, Y are randomly missing).  Use your preferred imputation method to fill in 10%, 20% and 30% of your missing data.

In [27]:
in_sample = bos.sample(frac=0.1, random_state=99)
out_sample = bos[~bos.isin(in_sample)].dropna()
in_sample['DIS'] = in_sample['DIS'].apply(lambda x: np.nan if in_sample.loc[in_sample['TAX'] > 300] else x)
in_sample['RAD'] = in_sample['RAD'].apply(lambda x: np.nan if in_sample.loc[in_sample['TAX'] > 300] else x)
in_sample['DIS'] = in_sample['DIS'].fillna(out_sample['DIS'].median())
in_sample['RAD'] = in_sample['RAD'].fillna(out_sample['DIS'].median())
imputed_data = pd.concat([in_sample, out_sample])
imputed_data = imputed_data.sort_index()

train_set = imputed_data.iloc[train_index]
test_set = imputed_data.iloc[test_index]

X_train = train_set.iloc[:, :-1].values
Y_train = train_set.iloc[:, -1].values
X_test = test_set.iloc[:, :-1].values
Y_test = test_set.iloc[:, -1].values

reg2 = LinearRegression().fit(X_train, Y_train)
print(reg2.score(X_train, Y_train))
print(reg2.coef_)
print(reg2.intercept_)
print(reg2.get_params())

Y_pred = reg2.predict(X_test)

mae = mean_absolute_error(Y_test,Y_pred)
mse = mean_squared_error(Y_test,Y_pred)
rmse_val = rmse(Y_test,Y_pred)
r2 = r2_score(Y_test,Y_pred)
print("MAE: %.3f"%mae)
print("MSE:  %.3f"%mse)
print("RMSE:  %.3f"%rmse_val)
print("R2:  %.3f"%r2)

temp_frame = pd.DataFrame({'data':'10% imputed, random conditional',
                   'imputation':'DIS, RAD',
                   'mae': mae, 
                   'mse': mse, 
                   'rmse':rmse_val,
                   'R2':r2,
                   'mae_diff':mae-orig_mae,
                   'mse_diff':mse-orig_mse,
                   'rmse_diff':rmse_val-orig_rmse_val,
                   'R2_diff':r2-orig_r2
                   }, index=[0])

res_frame = pd.concat([res_frame, temp_frame])
res_frame

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().